**Query Rewriter System**
**Overview**

This project implements a query rewriter system that takes a user query and rewrites it into a more personalized, human-readable format. The focus is on backend functionality and automatic evaluation rather than UI/UX.

The system can be used for:

Improving search results

Personalizing responses based on user queries

Enhancing conversational AI applications

**Dataset**

Synthetic Dataset: 200 sample queries generated from realistic topics like travel, food, tech, home improvement, and health.

Queries are structured to simulate typical user behavior and interests.

Example queries:

"Best Italian restaurants" → "Top-rated Italian restaurants near me"

"Repair washing machine" → "Step-by-step guide to repairing a washing machine"

**Query Rewriting Functionality**

The system uses google/flan-t5-large for rewriting queries.

Rewrites are concise, human-readable, and sometimes personalized based on context.

Queries are input via a live prompt in Colab, producing rewritten queries instantly.

**Evaluation Methodology**
Automatic Metrics
**bold text**
BLEU Score – Measures n-gram overlap between original and rewritten queries. Smoothed BLEU is used to handle short queries effectively.

ROUGE-L F1 – Measures longest common subsequence for quality comparison.

BERTScore – Uses contextual embeddings to evaluate semantic similarity between original and rewritten queries.

Qualitative/Heuristic Check

Queries are also checked for readability and minimum length (≥3 words).

Helps ensure rewrites are complete and meaningful.

Justification: BLEU and ROUGE are standard metrics for text similarity, while BERTScore captures semantic similarity. The heuristic ensures practical usefulness of the rewritten query.

**Setup Instructions**

Clone the repo:

git clone https://github.com/RUKESH-1704/Query-Rewriter/
cd Query-Rewriter


Install dependencies:

pip install -r requirements.txt


Or in Colab:

!pip install transformers datasets sentence-transformers rouge-score sacrebleu torch bert-score


Run the notebook or Python script to generate rewrites and compute evaluation metrics.

Enter queries in the live demo section to see rewrites and metrics.

**Outputs**

query_rewriter_outputs/all_rewrites.jsonl – All rewritten queries.

query_rewriter_outputs/automatic_metrics.json – BLEU, ROUGE-L, and BERTScore metrics.

query_rewriter_outputs/manual_check_samples.jsonl – Heuristic check samples.

In [1]:
!pip install transformers datasets sentence-transformers rouge-score sacrebleu torch bert-score --quiet

import os
import json
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
from bert_score import score as bert_score

os.makedirs("query_rewriter_outputs", exist_ok=True)

# Synthetic dataset

sample_queries = [
    "Best Italian restaurants", "Top sushi places", "Vegan cafe near me", "Affordable breakfast spots",
    "How to fix a leaky faucet", "DIY home plumbing repair", "Replace bathroom tiles", "Install shower head",
    "What are the benefits of meditation", "Mindfulness meditation techniques", "Yoga vs meditation benefits",
    "Guided meditation apps", "Latest movies", "Top Netflix movies", "Upcoming Marvel movies", "Oscar-winning films 2025",
    "How to lose weight fast", "Best diet plans for weight loss", "Home workouts for weight loss",
    "Low-carb meal recipes", "Cheap LED lights bulk", "Best energy-saving bulbs", "Buy smart bulbs online",
    "Laptop repair near me", "Replace laptop screen", "Fix slow computer", "Upgrade RAM for laptop",
    "Buy organic fertilizer", "Best fertilizers for home garden", "Compost vs organic fertilizer",
    "Keto diet meal plan", "Best keto snacks", "Keto recipes for beginners", "Keto meal prep ideas",
    "Solar panel subsidy eligibility", "Government solar panel schemes", "Install solar panels at home",
    "Flight from Delhi to Mumbai next week", "Cheap flights to New York", "Book train tickets online",
    "Best travel apps", "How to claim GST refund", "GST filing online", "GST refund process for business",
    "Best noise cancelling earbuds", "Top Bluetooth headphones", "Affordable wireless headphones",
    "How to treat fish fin rot", "Fish tank disease prevention", "Best aquarium care tips",
    "Doraemon movie", "Top anime movies", "Latest Disney movies", "Game of Thrones episodes",
    "Where to watch Game of Thrones online", "Repair washing machine", "Fix refrigerator leak",
    "Air conditioner troubleshooting", "School in India", "Top CBSE schools", "Best boarding schools",
    "Psychology books", "Best self-help books", "Top fiction novels", "Best AI books 2025",
    "Best gaming laptops 2025", "Top budget laptops", "Laptop buying guide", "Top ultrabooks 2025",
    "Top tourist places in Japan", "Best Europe travel destinations", "Adventure trips in India",
    "Affordable gym memberships", "Home workout routines", "Yoga classes near me",
    "How to learn Python programming", "Best coding courses online", "JavaScript tutorials",
    "Tips for improving sleep quality", "Best sleep tracking apps", "How to reduce insomnia",
    "Best smartphones under 300 USD", "Top mid-range smartphones", "Affordable flagship phones",
    "How to prepare for a job interview", "Resume writing tips", "Common interview questions",
    "DIY home decor ideas", "Budget interior decoration", "Creative wall painting ideas",
    "Best electric cars 2025", "Affordable electric scooters", "Top hybrid cars",
    "How to save money on groceries", "Budget meal planning", "Best cashback apps",
    "Yoga routines for beginners", "Morning yoga exercises", "Yoga poses for flexibility",
    "Cheap flights to London", "Book last-minute tickets", "Travel deals for students",
    "How to start a vegetable garden", "Urban gardening tips", "Best plants for balcony garden",
    "Best online courses for data science", "AI online courses", "Machine learning tutorials",
    "How to treat common cold", "Home remedies for flu", "Natural ways to boost immunity",
    "Top Netflix series to watch", "Best comedy shows", "Trending web series 2025",
    "How to apply for passport online", "Passport renewal process", "Travel visa requirements",
    "Affordable wedding venues", "Wedding planning checklist", "Best wedding decorators",
    "Best coffee shops in New York", "Top cafes in London", "Best brunch spots in Paris",
    "How to learn guitar quickly", "Guitar tutorials online", "Best beginner guitars",
    "Tips for effective time management", "Productivity apps", "Pomodoro technique guide",
    "Top-rated digital cameras", "Best mirrorless cameras", "Photography tips for beginners",
    "How to unclog a sink", "Drain cleaning hacks", "Plumber services near me",
    "Healthy meal prep ideas", "High-protein recipes", "Vegetarian meal plans",
    "Best hiking trails in California", "Adventure travel spots", "Top trekking destinations",
    "How to improve memory", "Brain exercises", "Memory improvement tips",
    "Affordable skincare products", "Best moisturizers", "Anti-aging creams",
    "Top fantasy books of 2025", "Science fiction novels", "Best mystery books",
    "How to set up a home office", "Ergonomic office setup", "Work from home tips",
    "Cheap hotels in Paris", "Budget travel accommodation", "Top-rated hostels",
    "How to start a podcast", "Podcast hosting platforms", "Podcast recording tips",
    "Best indoor plants for home", "Air-purifying plants", "Easy to maintain plants",
    "Tips for reducing stress", "Meditation for stress relief", "Mindfulness exercises",
    "How to improve public speaking", "Toastmasters tips", "Presentation skills guide",
    "Top-rated smartwatches", "Best fitness watches", "Affordable smartwatches",
    "How to train for a marathon", "Running plans for beginners", "Marathon diet tips",
    "Best budget laptops", "Laptop deals online", "Gaming laptops under $1000",
    "How to repair smartphone screen", "Replace iPhone screen", "Fix Android screen crack",
    "Top board games for family", "Best card games", "Educational board games",
    "How to start freelancing", "Freelance platforms", "Tips to get first client",
    "Best travel backpacks", "Hiking backpacks", "School bags for students",
    "How to meditate effectively", "Mindfulness meditation tips", "Guided meditation apps",
    "Healthy snacks for kids", "Easy snack recipes", "Nutritious lunchbox ideas",
    "How to create a budget plan", "Personal finance tips", "Expense tracking apps",
    "Top-rated restaurants in London", "Best sushi restaurants", "Fine dining options",
    "How to install solar panels", "Home solar installation", "Solar energy tips",
    "Best coding bootcamps", "Online coding bootcamps", "Python programming courses",
    "How to lose belly fat", "Fat burning exercises", "Diet plans for flat stomach",
    "Top-rated air purifiers", "Best HEPA filters", "Home air quality tips",
    "How to fix WiFi connection", "Improve internet speed", "Router troubleshooting",
    "Best books for self-improvement", "Personal growth books", "Motivational books",
    "How to start YouTube channel", "YouTube content ideas", "Video editing tips",
    "Top fantasy movies", "Best action films", "Animated movies to watch",
    "How to declutter home", "Minimalist living tips", "Home organization hacks",
    "Affordable interior design tips", "Budget-friendly decor", "Small apartment design ideas",
    "Best running shoes", "Top sports shoes", "Trail running shoes",
    "How to train a dog", "Dog obedience training", "Potty training tips",
    "Top-rated hotels in Tokyo", "Best luxury hotels", "Affordable stays in Tokyo",
    "How to make homemade pizza", "Pizza recipes", "Easy dough preparation",
    "Healthy smoothie recipes", "Green smoothie recipes", "Protein smoothie ideas",
    "How to learn Spanish quickly", "Language learning apps", "Spanish grammar tips",
    "Best headphones for gaming", "Noise cancelling headphones", "Wireless gaming headsets",
    "How to reduce electricity bill", "Energy saving tips", "Home energy audit",
    "Top tourist attractions in India", "Historical monuments in India", "Must-visit temples",
    "How to clean carpets", "Carpet cleaning tips", "DIY floor cleaning",
    "Best budget smartphones", "Top Android phones", "Affordable iPhones",
    "How to write a resume", "Resume format tips", "Cover letter examples",
    "Top Netflix movies", "Trending shows", "Best documentaries",
    "How to lose weight naturally", "Healthy eating plan", "Fitness routines at home",
    "Best online marketplaces", "Top e-commerce platforms", "Cheap online deals",
    "How to repair bicycle", "Fix bike chain", "Replace bicycle tire",
    "Top-rated restaurants in Paris", "Fine dining Paris", "French cuisine restaurants",
    "How to plan a trip to Italy", "Italy travel itinerary", "Rome tourist guide"
]

synthetic_queries = [{"query": q} for q in random.choices(sample_queries, k=200)]
dataset = Dataset.from_list(synthetic_queries)

# Load model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

# Query rewriting function
def rewrite_query(query):
    prompt = f"""
You are a query rewriting assistant.
Rewrite the following user query into a concise, human-readable query.
Examples:
Original: Best Italian restaurants
Rewritten: Top-rated Italian restaurants near me
Original: How to fix a leaky faucet
Rewritten: Step-by-step guide to fixing a faucet leak
Original: What are the benefits of meditation
Rewritten: Benefits of mindfulness meditation for stress relief
Original: Latest movies
Rewritten: New releases in cinema this year
Original: How to lose weight fast
Rewritten: Effective weight loss strategies
Original: Doraemon movie
Rewritten: Latest Doraemon movies to watch
Original: Game of Thrones episodes
Rewritten: Where to watch Game of Thrones online
Original: Repair washing machine
Rewritten: Step-by-step guide to repairing a washing machine

Now rewrite this query:
Original: {query}
Rewritten:
"""
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs['input_ids'],
        max_length=80,
        num_beams=5,
        temperature=0.7,
        early_stopping=True
    )
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if len(rewritten.strip()) < 5:
        rewritten = f"Optimized search: {query}"
    return rewritten

# Rewrite all dataset queries
rewrites = []
for item in dataset:
    rewritten = rewrite_query(item['query'])
    rewrites.append({
        "original_query": item['query'],
        "rewritten_query": rewritten
    })

with open("query_rewriter_outputs/all_rewrites.jsonl", "w") as f:
    for r in rewrites:
        f.write(json.dumps(r) + "\n")

# Automatic evaluation
references = [r['original_query'] for r in rewrites]
hypotheses = [r['rewritten_query'] for r in rewrites]

bleu_score = corpus_bleu(hypotheses, [references]).score
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_f1 = sum([rouge.score(h, r)['rougeL'].fmeasure for h, r in zip(hypotheses, references)]) / len(hypotheses)
P, R, F1 = bert_score(hypotheses, references, lang='en')

metrics = {
    "BLEU": bleu_score,
    "ROUGE-L_F1": rouge_l_f1,
    "BERTScore_F1_mean": F1.mean().item()
}

# Save to file
with open("query_rewriter_outputs/automatic_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print("\n=== Automatic Evaluation Metrics ===")
print(f"BLEU Score       : {metrics['BLEU']:.2f}")
print(f"ROUGE-L F1 Score : {metrics['ROUGE-L_F1']:.2f}")
print(f"BERTScore F1 Avg : {metrics['BERTScore_F1_mean']:.2f}")

heuristics = []
for r in rewrites:
    passes_length_check = len(r['rewritten_query'].split()) >= 3
    heuristics.append({
        "original_query": r['original_query'],
        "rewritten_query": r['rewritten_query'],
        "passes_length_check": passes_length_check
    })

with open("query_rewriter_outputs/manual_check_samples.jsonl", "w") as f:
    for h in heuristics[:50]:
        f.write(json.dumps(h) + "\n")

while True:
    user_query = input("Enter your query (or 'exit' to quit): ")
    if user_query.lower() == "exit":
        break
    rewritten_query = rewrite_query(user_query)
    print("Original query:", user_query)
    print("Rewritten query:", rewritten_query)
    print()


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter your query (or 'exit' to quit): repair grinder
Original query: repair grinder
Rewritten query: how do you repair a grinder?

Enter your query (or 'exit' to quit): hotels near
Original query: hotels near
Rewritten query: find hotels near me

Enter your query (or 'exit' to quit): exit


In [3]:
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

while True:
    user_query = input("Enter your query (or 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    rewritten_query = rewrite_query(user_query)

    bleu = corpus_bleu([rewritten_query], [[user_query]]).score
    rouge_l_f1 = rouge.score(rewritten_query, user_query)['rougeL'].fmeasure
    P, R, F1 = bert_score([rewritten_query], [user_query], lang='en')
    bert_f1 = F1.mean().item()

    print("\nOriginal query       :", user_query)
    print("Rewritten query      :", rewritten_query)
    print(f"BLEU Score           : {bleu:.2f}")
    print(f"ROUGE-L F1 Score     : {rouge_l_f1:.2f}")
    print(f"BERTScore F1         : {bert_f1:.2f}\n")


Enter your query (or 'exit' to quit): grinder repair


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Original query       : grinder repair
Rewritten query      : how do you repair a grinder?
BLEU Score           : 7.81
ROUGE-L F1 Score     : 0.25
BERTScore F1         : 0.85

Enter your query (or 'exit' to quit): best hotels


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Original query       : best hotels
Rewritten query      : What are the best hotels near me?
BLEU Score           : 11.04
ROUGE-L F1 Score     : 0.44
BERTScore F1         : 0.88

Enter your query (or 'exit' to quit): india population


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Original query       : india population
Rewritten query      : what is the population of india
BLEU Score           : 9.65
ROUGE-L F1 Score     : 0.25
BERTScore F1         : 0.89

Enter your query (or 'exit' to quit): exit


In [5]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

smooth_fn = SmoothingFunction().method2

while True:
    user_query = input("Enter your query (or 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    rewritten_query = rewrite_query(user_query)

    # BLEU
    bleu_score_val = corpus_bleu(
        [[user_query.split()]],
        [rewritten_query.split()],
        smoothing_function=smooth_fn
    ) * 100

    # ROUGE-L
    rouge_l_f1 = rouge.score(rewritten_query, user_query)['rougeL'].fmeasure

    # BERTScore
    P, R, F1 = bert_score([rewritten_query], [user_query], lang='en')

    print("\nOriginal query       :", user_query)
    print("Rewritten query      :", rewritten_query)
    print(f"BLEU Score           : {bleu_score_val:.2f}")
    print(f"ROUGE-L F1 Score     : {rouge_l_f1:.2f}")
    print(f"BERTScore F1         : {F1.mean().item():.2f}\n")


Enter your query (or 'exit' to quit): best movies 


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Original query       : best movies 
Rewritten query      : What are the best movies?
BLEU Score           : 24.03
ROUGE-L F1 Score     : 0.57
BERTScore F1         : 0.89

Enter your query (or 'exit' to quit): hotels near


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Original query       : hotels near
Rewritten query      : find hotels near me
BLEU Score           : 45.18
ROUGE-L F1 Score     : 0.67
BERTScore F1         : 0.88

Enter your query (or 'exit' to quit): exit
